<a href="https://colab.research.google.com/github/siwarnasri/MlOps_CustomerSatisfaction/blob/main/1_1_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 1.1: ML Pipelines

In this notebook, you will learn how to convert existing ML code into ML pipelines using ZenML.

Since we will be creating models using Sklearn, you must have the ZenML Sklearn integration installed. You can install ZenML and the Sklearn integration with the following command, which will also reboot your notebook's kernel.

In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00

**Colab Note:** On Colab, you need an [ngrok account](https://dashboard.ngrok.com/signup) to view some of the visualizations later. Please set up an account, then set your user token below:

In [ ]:
NGROK_TOKEN = "ONNKZJSBB6C24BGK2DLSM5C4MQDORS5V"  # TODO: set your ngrok token if you are working on Colab

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab only setup

    # install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

As an ML practitioner, you are probably familiar with building ML models using Scikit-learn, PyTorch, TensorFlow, or similar. An ML pipeline is simply an extension that includes other steps you would normally perform before or after creating a model, such as data collection, preprocessing, model deployment, or monitoring. The ML pipeline essentially defines a step-by-step process for your work as an ML practitioner. Defining ML pipelines explicitly in code is great because:

We can easily repeat all of our work, not just the model, to eliminate errors and make our models easier to reproduce.
Data and models can be versioned and tracked, so we can see at a glance which dataset a model was trained on and how it compares to other models.
When the entire pipeline is coded, we can automate many operational tasks, such as re-training and redeploying models when the underlying problem or data changes, or rolling out new and improved models with CI/CD workflows.
A well-defined ML pipeline is essential for ML teams looking to deploy models at scale.

## ZenML Setup
Throughout this series, we will define our ML pipelines using [ZenML](https://github.com/zenml-io/zenml/). ZenML is an excellent tool for this task, as it is easy and intuitive to use, and has [integrations](https://zenml.io/integrations) with most of the advanced MLOps tools we will use later. Make sure you have ZenML installed (via `pip install zenml`). Next, we run some commands to make sure you start with a fresh ML stack.